In [4]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
from datetime import datetime
folder_path = os.getcwd()
# Read the name of all excel files
excel_files = glob.glob(os.path.join(folder_path, "*.xlsx"))

In [5]:
# Read the excel file name
folder_path = os.getcwd()
excel_files = [file for file in os.listdir(folder_path) if file.endswith('.xlsx')]

# Read the file and drop the na rows
dataframes = {}
for file in excel_files:
    print(file)
    df = pd.read_excel(os.path.join(folder_path, file), engine='openpyxl')
#     df.dropna(subset=['ClosePrice'], inplace=True)
    dataframes[file[:-5]] = df

TCL中环_results1_1day.xlsx
三峡能源_results1_1day.xlsx
中国广核_results1_1day.xlsx
中国核电_results1_1day.xlsx
亿纬锂能_results1_1day.xlsx
华友钴业_results1_1day.xlsx
大全能源_results1_1day.xlsx
天合光能_results1_1day.xlsx
天赐材料_results1_1day.xlsx
天齐锂业_results1_1day.xlsx
宁德时代_results1_1day.xlsx
恩捷股份_results1_1day.xlsx
晶澳科技_results1_1day.xlsx
晶盛机电_results1_1day.xlsx
福斯特_results1_1day.xlsx
赣锋锂业_results1_1day.xlsx
通威股份_results1_1day.xlsx
阳光电源_results1_1day.xlsx
隆基绿能_results1_1day.xlsx
龙源电力_results1_1day.xlsx


In [6]:
stock_list = list(dataframes.keys())
stock_list = [i[:-14] for i in stock_list]
stock_list

['TCL中环',
 '三峡能源',
 '中国广核',
 '中国核电',
 '亿纬锂能',
 '华友钴业',
 '大全能源',
 '天合光能',
 '天赐材料',
 '天齐锂业',
 '宁德时代',
 '恩捷股份',
 '晶澳科技',
 '晶盛机电',
 '福斯特',
 '赣锋锂业',
 '通威股份',
 '阳光电源',
 '隆基绿能',
 '龙源电力']

In [7]:
#Settle the trading structure
class StockTradingFramework:
    def __init__(self, initial_capital):
        self.capital = initial_capital
        self.positions = pd.DataFrame(columns=['symbol', 'position_type', 'entry_time', 'entry_price', 'quantity'])
        self.trades = pd.DataFrame(columns=['symbol', 'position_type', 'entry_time', 'exit_time', 'quantity', 'profit', 'return'])

    def enter_position(self, symbol, position_type, entry_time, entry_price, quantity):
        if position_type not in ['long', 'short']:
            raise ValueError('position_type should be either "long" or "short"')
        if entry_price <= 0 or quantity <= 0:
            raise ValueError('entry_price and quantity should be positive numbers')
        if self.capital < entry_price * quantity:
            raise ValueError('not enough capital to enter position')
        
        new_position = pd.DataFrame({'symbol': symbol, 'position_type': position_type, 'entry_time': entry_time, 'entry_price': entry_price, 'quantity': quantity}, index=[0])
        self.positions = pd.concat([self.positions, new_position], ignore_index=True)

        self.capital -= entry_price * quantity
        self.positions = self.positions.reset_index(drop = True)

    def exit_position(self, symbol, position_type, exit_time, exit_price):
        position = self.positions[(self.positions['symbol'] == symbol) & (self.positions['position_type'] == position_type)].iloc[:1]
        if position.empty:
            raise ValueError('no open position found')
        entry_time = position.iloc[0]['entry_time']
        entry_price = position.iloc[0]['entry_price']
        quantity = position.iloc[0]['quantity']
        profit = (exit_price - entry_price) * quantity if position_type == 'long' else (entry_price - exit_price) * quantity
        return_rate = profit / (entry_price * quantity)
        
        new_trade = pd.DataFrame({'symbol': symbol, 'position_type': position_type, 'entry_time': entry_time, 'exit_time': exit_time, 'entry_price': entry_price, 'exit_price': exit_price ,'quantity': quantity, 'profit': profit, 'return': return_rate}, index=[0])
        self.trades = pd.concat([self.trades, new_trade], ignore_index=True)

        self.capital += exit_price * quantity
        self.positions.loc[position.index, 'exit_time'] = exit_time
        self.positions.loc[position.index, 'exit_price'] = exit_price
        self.positions = self.positions[self.positions['exit_time'].isnull()]
        self.positions = self.positions.reset_index(drop = True)
    def get_positions(self):
        return self.positions

    def get_available_capital(self):
        return self.capital

    def get_trades(self, position_type=None):
        if position_type is None:
            return self.trades
        else:
            return self.trades[self.trades['position_type'] == position_type]

    def plot_profit(self):
        long_trades = self.get_trades('long')
        short_trades = self.get_trades('short')
        profit = pd.concat([long_trades, short_trades]).groupby('exit_time')['profit'].sum().cumsum()
        plt.plot(profit)
        plt.title('Cumulative Profit')
        plt.xlabel('Time')
        plt.ylabel('Profit')
        plt.show()

In [8]:
stock_data = dataframes['TCL中环_results1_1day']
stock_data = stock_data.reset_index(drop = True)
stock_data['Date_new'] = stock_data['date']
stock_data.set_index('Date_new',inplace = True)
stock_data['Predicted'] = stock_data['Predicted price'] / stock_data['open'] - 1

In [14]:
#define the funtion tio simulate the trade for a certain stock
def trading_LSTM(initial_capital,stock_name,stock_data,step,threshold,rf):

    #Read the Stock Data From Dataframes(dict)
    stock_data = dataframes[stock_name+'_results1_1day']
    stock_data = stock_data.reset_index(drop = True)
    stock_data['Date_new'] = stock_data['date']
    stock_data.set_index('Date_new',inplace = True)
    stock_data['Predicted'] = stock_data['Predicted price'] / stock_data['open'] - 1
    
    #Get the list if trading date
    date_list = stock_data['date'].unique().tolist()
    date_l = []
    for date in date_list:
        dt = datetime.fromtimestamp(int(date / 1e9))
        dt = dt.strftime('%Y-%m-%d')
        date_l.append(dt)
        
    #Initialize the current_date's index, position_status and capital
    day_index = 0
    holding_status = 0
    framework = StockTradingFramework(initial_capital=initial_capital)
    
    #Start trading
    while day_index < len(date_l):
        current_date = date_l[day_index]
        predicted_return = stock_data.loc[current_date,'Predicted']
        stock_open_price = stock_data.loc[current_date,'open']

        if predicted_return > threshold and holding_status == 0:
            current_capital = framework.get_available_capital()
            available_quantity = int(current_capital / stock_open_price)
            framework.enter_position(symbol = stock_name, position_type = 'long', \
                                     entry_time = current_date, entry_price = stock_open_price, quantity = available_quantity)
            holding_status = 1
        elif predicted_return < (-1)*threshold and holding_status == 1:
            framework.exit_position(symbol=stock_name, position_type = 'long',exit_time=current_date, exit_price=stock_open_price)
            holding_status = 0

        if ((day_index+step) >= len(date_l)) and (holding_status == 1):
            framework.exit_position(symbol=stock_name, position_type = 'long',exit_time=current_date, exit_price=stock_open_price)

        day_index += step
        
    final_total_asset = framework.get_available_capital()
    
    mature_return = final_total_asset / 100000 - 1
    annual_return = mature_return*4
    volatility = framework.get_trades()['return'].std()
    sharp_ratio = (annual_return - rf)/volatility
    largest_back = framework.get_trades()['return'].min()
    Tradingcounts = framework.get_trades().shape[0]+framework.get_positions().shape[0]
    
    result = pd.DataFrame({'stock_name':[stock_name],'mature_return':[mature_return],'volatility':[volatility],'sharp_ratio':[sharp_ratio],\
                           'step':[step],'rf':[rf],'threshold':[threshold],'largest_back':[largest_back],'Tradingcounts':[Tradingcounts]})  
    return result
    
        

In [15]:
# get the risk free rate
rf_df = pd.read_excel('C:/Users/周捷骋/Desktop/Trading_data_new/indicators/Rf.xlsx')
rf_df['日期'] = pd.to_datetime(rf_df['日期'])
rf_df['Rf'] = rf_df['Rf']/100
rf_df = rf_df[(rf_df['日期'] >= '2023-01-01') & (rf_df['日期'] <= '2023-03-31')]
avg_rf =rf_df['Rf'].mean()

In [17]:
#start trading and recording the result
trading_record = pd.DataFrame()
for i in range(20):
    stock_name = stock_list[i]
    for n in range(14):
        for k in range(50):
            threshold = (k+1)/1000
            print(f'Stock: {stock_name},Trading_Step: {n+1}, Threshold: {threshold}')
            a = trading_LSTM(100000,stock_name,dataframes,n+1,threshold,avg_rf)
            trading_record = pd.concat([a,trading_record])

Stock: TCL中环,Trading_Step: 1, Threshold: 0.001
Stock: TCL中环,Trading_Step: 1, Threshold: 0.002
Stock: TCL中环,Trading_Step: 1, Threshold: 0.003
Stock: TCL中环,Trading_Step: 1, Threshold: 0.004
Stock: TCL中环,Trading_Step: 1, Threshold: 0.005
Stock: TCL中环,Trading_Step: 1, Threshold: 0.006
Stock: TCL中环,Trading_Step: 1, Threshold: 0.007
Stock: TCL中环,Trading_Step: 1, Threshold: 0.008
Stock: TCL中环,Trading_Step: 1, Threshold: 0.009
Stock: TCL中环,Trading_Step: 1, Threshold: 0.01
Stock: TCL中环,Trading_Step: 1, Threshold: 0.011
Stock: TCL中环,Trading_Step: 1, Threshold: 0.012
Stock: TCL中环,Trading_Step: 1, Threshold: 0.013
Stock: TCL中环,Trading_Step: 1, Threshold: 0.014
Stock: TCL中环,Trading_Step: 1, Threshold: 0.015
Stock: TCL中环,Trading_Step: 1, Threshold: 0.016
Stock: TCL中环,Trading_Step: 1, Threshold: 0.017
Stock: TCL中环,Trading_Step: 1, Threshold: 0.018
Stock: TCL中环,Trading_Step: 1, Threshold: 0.019
Stock: TCL中环,Trading_Step: 1, Threshold: 0.02
Stock: TCL中环,Trading_Step: 1, Threshold: 0.021
Stock: TCL中环,Tr

Stock: TCL中环,Trading_Step: 4, Threshold: 0.038
Stock: TCL中环,Trading_Step: 4, Threshold: 0.039
Stock: TCL中环,Trading_Step: 4, Threshold: 0.04
Stock: TCL中环,Trading_Step: 4, Threshold: 0.041
Stock: TCL中环,Trading_Step: 4, Threshold: 0.042
Stock: TCL中环,Trading_Step: 4, Threshold: 0.043
Stock: TCL中环,Trading_Step: 4, Threshold: 0.044
Stock: TCL中环,Trading_Step: 4, Threshold: 0.045
Stock: TCL中环,Trading_Step: 4, Threshold: 0.046
Stock: TCL中环,Trading_Step: 4, Threshold: 0.047
Stock: TCL中环,Trading_Step: 4, Threshold: 0.048
Stock: TCL中环,Trading_Step: 4, Threshold: 0.049
Stock: TCL中环,Trading_Step: 4, Threshold: 0.05
Stock: TCL中环,Trading_Step: 5, Threshold: 0.001
Stock: TCL中环,Trading_Step: 5, Threshold: 0.002
Stock: TCL中环,Trading_Step: 5, Threshold: 0.003
Stock: TCL中环,Trading_Step: 5, Threshold: 0.004
Stock: TCL中环,Trading_Step: 5, Threshold: 0.005
Stock: TCL中环,Trading_Step: 5, Threshold: 0.006
Stock: TCL中环,Trading_Step: 5, Threshold: 0.007
Stock: TCL中环,Trading_Step: 5, Threshold: 0.008
Stock: TCL中环,Tr

Stock: TCL中环,Trading_Step: 8, Threshold: 0.029
Stock: TCL中环,Trading_Step: 8, Threshold: 0.03
Stock: TCL中环,Trading_Step: 8, Threshold: 0.031
Stock: TCL中环,Trading_Step: 8, Threshold: 0.032
Stock: TCL中环,Trading_Step: 8, Threshold: 0.033
Stock: TCL中环,Trading_Step: 8, Threshold: 0.034
Stock: TCL中环,Trading_Step: 8, Threshold: 0.035
Stock: TCL中环,Trading_Step: 8, Threshold: 0.036
Stock: TCL中环,Trading_Step: 8, Threshold: 0.037
Stock: TCL中环,Trading_Step: 8, Threshold: 0.038
Stock: TCL中环,Trading_Step: 8, Threshold: 0.039
Stock: TCL中环,Trading_Step: 8, Threshold: 0.04
Stock: TCL中环,Trading_Step: 8, Threshold: 0.041
Stock: TCL中环,Trading_Step: 8, Threshold: 0.042
Stock: TCL中环,Trading_Step: 8, Threshold: 0.043
Stock: TCL中环,Trading_Step: 8, Threshold: 0.044
Stock: TCL中环,Trading_Step: 8, Threshold: 0.045
Stock: TCL中环,Trading_Step: 8, Threshold: 0.046
Stock: TCL中环,Trading_Step: 8, Threshold: 0.047
Stock: TCL中环,Trading_Step: 8, Threshold: 0.048
Stock: TCL中环,Trading_Step: 8, Threshold: 0.049
Stock: TCL中环,Tr

Stock: TCL中环,Trading_Step: 12, Threshold: 0.011
Stock: TCL中环,Trading_Step: 12, Threshold: 0.012
Stock: TCL中环,Trading_Step: 12, Threshold: 0.013
Stock: TCL中环,Trading_Step: 12, Threshold: 0.014
Stock: TCL中环,Trading_Step: 12, Threshold: 0.015
Stock: TCL中环,Trading_Step: 12, Threshold: 0.016
Stock: TCL中环,Trading_Step: 12, Threshold: 0.017
Stock: TCL中环,Trading_Step: 12, Threshold: 0.018
Stock: TCL中环,Trading_Step: 12, Threshold: 0.019
Stock: TCL中环,Trading_Step: 12, Threshold: 0.02
Stock: TCL中环,Trading_Step: 12, Threshold: 0.021
Stock: TCL中环,Trading_Step: 12, Threshold: 0.022
Stock: TCL中环,Trading_Step: 12, Threshold: 0.023
Stock: TCL中环,Trading_Step: 12, Threshold: 0.024
Stock: TCL中环,Trading_Step: 12, Threshold: 0.025
Stock: TCL中环,Trading_Step: 12, Threshold: 0.026
Stock: TCL中环,Trading_Step: 12, Threshold: 0.027
Stock: TCL中环,Trading_Step: 12, Threshold: 0.028
Stock: TCL中环,Trading_Step: 12, Threshold: 0.029
Stock: TCL中环,Trading_Step: 12, Threshold: 0.03
Stock: TCL中环,Trading_Step: 12, Threshold: 

Stock: 三峡能源,Trading_Step: 1, Threshold: 0.035
Stock: 三峡能源,Trading_Step: 1, Threshold: 0.036
Stock: 三峡能源,Trading_Step: 1, Threshold: 0.037
Stock: 三峡能源,Trading_Step: 1, Threshold: 0.038
Stock: 三峡能源,Trading_Step: 1, Threshold: 0.039
Stock: 三峡能源,Trading_Step: 1, Threshold: 0.04
Stock: 三峡能源,Trading_Step: 1, Threshold: 0.041
Stock: 三峡能源,Trading_Step: 1, Threshold: 0.042
Stock: 三峡能源,Trading_Step: 1, Threshold: 0.043
Stock: 三峡能源,Trading_Step: 1, Threshold: 0.044
Stock: 三峡能源,Trading_Step: 1, Threshold: 0.045
Stock: 三峡能源,Trading_Step: 1, Threshold: 0.046
Stock: 三峡能源,Trading_Step: 1, Threshold: 0.047
Stock: 三峡能源,Trading_Step: 1, Threshold: 0.048
Stock: 三峡能源,Trading_Step: 1, Threshold: 0.049
Stock: 三峡能源,Trading_Step: 1, Threshold: 0.05
Stock: 三峡能源,Trading_Step: 2, Threshold: 0.001
Stock: 三峡能源,Trading_Step: 2, Threshold: 0.002
Stock: 三峡能源,Trading_Step: 2, Threshold: 0.003
Stock: 三峡能源,Trading_Step: 2, Threshold: 0.004
Stock: 三峡能源,Trading_Step: 2, Threshold: 0.005
Stock: 三峡能源,Trading_Step: 2, Thresho

...

Stock: 隆基绿能,Trading_Step: 12, Threshold: 0.011
Stock: 隆基绿能,Trading_Step: 12, Threshold: 0.012
Stock: 隆基绿能,Trading_Step: 12, Threshold: 0.013
Stock: 隆基绿能,Trading_Step: 12, Threshold: 0.014
Stock: 隆基绿能,Trading_Step: 12, Threshold: 0.015
Stock: 隆基绿能,Trading_Step: 12, Threshold: 0.016
Stock: 隆基绿能,Trading_Step: 12, Threshold: 0.017
Stock: 隆基绿能,Trading_Step: 12, Threshold: 0.018
Stock: 隆基绿能,Trading_Step: 12, Threshold: 0.019
Stock: 隆基绿能,Trading_Step: 12, Threshold: 0.02
Stock: 隆基绿能,Trading_Step: 12, Threshold: 0.021
Stock: 隆基绿能,Trading_Step: 12, Threshold: 0.022
Stock: 隆基绿能,Trading_Step: 12, Threshold: 0.023
Stock: 隆基绿能,Trading_Step: 12, Threshold: 0.024
Stock: 隆基绿能,Trading_Step: 12, Threshold: 0.025
Stock: 隆基绿能,Trading_Step: 12, Threshold: 0.026
Stock: 隆基绿能,Trading_Step: 12, Threshold: 0.027
Stock: 隆基绿能,Trading_Step: 12, Threshold: 0.028
Stock: 隆基绿能,Trading_Step: 12, Threshold: 0.029
Stock: 隆基绿能,Trading_Step: 12, Threshold: 0.03
Stock: 隆基绿能,Trading_Step: 12, Threshold: 0.031
Stock: 隆基绿能,Tra

Stock: 龙源电力,Trading_Step: 1, Threshold: 0.04
Stock: 龙源电力,Trading_Step: 1, Threshold: 0.041
Stock: 龙源电力,Trading_Step: 1, Threshold: 0.042
Stock: 龙源电力,Trading_Step: 1, Threshold: 0.043
Stock: 龙源电力,Trading_Step: 1, Threshold: 0.044
Stock: 龙源电力,Trading_Step: 1, Threshold: 0.045
Stock: 龙源电力,Trading_Step: 1, Threshold: 0.046
Stock: 龙源电力,Trading_Step: 1, Threshold: 0.047
Stock: 龙源电力,Trading_Step: 1, Threshold: 0.048
Stock: 龙源电力,Trading_Step: 1, Threshold: 0.049
Stock: 龙源电力,Trading_Step: 1, Threshold: 0.05
Stock: 龙源电力,Trading_Step: 2, Threshold: 0.001
Stock: 龙源电力,Trading_Step: 2, Threshold: 0.002
Stock: 龙源电力,Trading_Step: 2, Threshold: 0.003
Stock: 龙源电力,Trading_Step: 2, Threshold: 0.004
Stock: 龙源电力,Trading_Step: 2, Threshold: 0.005
Stock: 龙源电力,Trading_Step: 2, Threshold: 0.006
Stock: 龙源电力,Trading_Step: 2, Threshold: 0.007
Stock: 龙源电力,Trading_Step: 2, Threshold: 0.008
Stock: 龙源电力,Trading_Step: 2, Threshold: 0.009
Stock: 龙源电力,Trading_Step: 2, Threshold: 0.01
Stock: 龙源电力,Trading_Step: 2, Threshol

Stock: 龙源电力,Trading_Step: 5, Threshold: 0.023
Stock: 龙源电力,Trading_Step: 5, Threshold: 0.024
Stock: 龙源电力,Trading_Step: 5, Threshold: 0.025
Stock: 龙源电力,Trading_Step: 5, Threshold: 0.026
Stock: 龙源电力,Trading_Step: 5, Threshold: 0.027
Stock: 龙源电力,Trading_Step: 5, Threshold: 0.028
Stock: 龙源电力,Trading_Step: 5, Threshold: 0.029
Stock: 龙源电力,Trading_Step: 5, Threshold: 0.03
Stock: 龙源电力,Trading_Step: 5, Threshold: 0.031
Stock: 龙源电力,Trading_Step: 5, Threshold: 0.032
Stock: 龙源电力,Trading_Step: 5, Threshold: 0.033
Stock: 龙源电力,Trading_Step: 5, Threshold: 0.034
Stock: 龙源电力,Trading_Step: 5, Threshold: 0.035
Stock: 龙源电力,Trading_Step: 5, Threshold: 0.036
Stock: 龙源电力,Trading_Step: 5, Threshold: 0.037
Stock: 龙源电力,Trading_Step: 5, Threshold: 0.038
Stock: 龙源电力,Trading_Step: 5, Threshold: 0.039
Stock: 龙源电力,Trading_Step: 5, Threshold: 0.04
Stock: 龙源电力,Trading_Step: 5, Threshold: 0.041
Stock: 龙源电力,Trading_Step: 5, Threshold: 0.042
Stock: 龙源电力,Trading_Step: 5, Threshold: 0.043
Stock: 龙源电力,Trading_Step: 5, Thresho

Stock: 龙源电力,Trading_Step: 9, Threshold: 0.016
Stock: 龙源电力,Trading_Step: 9, Threshold: 0.017
Stock: 龙源电力,Trading_Step: 9, Threshold: 0.018
Stock: 龙源电力,Trading_Step: 9, Threshold: 0.019
Stock: 龙源电力,Trading_Step: 9, Threshold: 0.02
Stock: 龙源电力,Trading_Step: 9, Threshold: 0.021
Stock: 龙源电力,Trading_Step: 9, Threshold: 0.022
Stock: 龙源电力,Trading_Step: 9, Threshold: 0.023
Stock: 龙源电力,Trading_Step: 9, Threshold: 0.024
Stock: 龙源电力,Trading_Step: 9, Threshold: 0.025
Stock: 龙源电力,Trading_Step: 9, Threshold: 0.026
Stock: 龙源电力,Trading_Step: 9, Threshold: 0.027
Stock: 龙源电力,Trading_Step: 9, Threshold: 0.028
Stock: 龙源电力,Trading_Step: 9, Threshold: 0.029
Stock: 龙源电力,Trading_Step: 9, Threshold: 0.03
Stock: 龙源电力,Trading_Step: 9, Threshold: 0.031
Stock: 龙源电力,Trading_Step: 9, Threshold: 0.032
Stock: 龙源电力,Trading_Step: 9, Threshold: 0.033
Stock: 龙源电力,Trading_Step: 9, Threshold: 0.034
Stock: 龙源电力,Trading_Step: 9, Threshold: 0.035
Stock: 龙源电力,Trading_Step: 9, Threshold: 0.036
Stock: 龙源电力,Trading_Step: 9, Thresho

Stock: 龙源电力,Trading_Step: 13, Threshold: 0.005
Stock: 龙源电力,Trading_Step: 13, Threshold: 0.006
Stock: 龙源电力,Trading_Step: 13, Threshold: 0.007
Stock: 龙源电力,Trading_Step: 13, Threshold: 0.008
Stock: 龙源电力,Trading_Step: 13, Threshold: 0.009
Stock: 龙源电力,Trading_Step: 13, Threshold: 0.01
Stock: 龙源电力,Trading_Step: 13, Threshold: 0.011
Stock: 龙源电力,Trading_Step: 13, Threshold: 0.012
Stock: 龙源电力,Trading_Step: 13, Threshold: 0.013
Stock: 龙源电力,Trading_Step: 13, Threshold: 0.014
Stock: 龙源电力,Trading_Step: 13, Threshold: 0.015
Stock: 龙源电力,Trading_Step: 13, Threshold: 0.016
Stock: 龙源电力,Trading_Step: 13, Threshold: 0.017
Stock: 龙源电力,Trading_Step: 13, Threshold: 0.018
Stock: 龙源电力,Trading_Step: 13, Threshold: 0.019
Stock: 龙源电力,Trading_Step: 13, Threshold: 0.02
Stock: 龙源电力,Trading_Step: 13, Threshold: 0.021
Stock: 龙源电力,Trading_Step: 13, Threshold: 0.022
Stock: 龙源电力,Trading_Step: 13, Threshold: 0.023
Stock: 龙源电力,Trading_Step: 13, Threshold: 0.024
Stock: 龙源电力,Trading_Step: 13, Threshold: 0.025
Stock: 龙源电力,Tra

In [18]:
trading_record.sort_values(by = 'mature_return',ascending = False)

,stock_name,mature_return,volatility,sharp_ratio,step,rf,threshold,largest_back,Tradingcounts
0,TCL中环,0.501499,0.029441,66.638842,1,0.044106,0.047,0.124160,3
0,TCL中环,0.501499,0.029441,66.638842,1,0.044106,0.048,0.124160,3
0,TCL中环,0.501499,0.029441,66.638842,1,0.044106,0.049,0.124160,3
0,TCL中环,0.469889,0.116292,15.783121,2,0.044106,0.038,0.133099,2
0,TCL中环,0.469889,0.116292,15.783121,2,0.044106,0.047,0.133099,2
...,...,...,...,...,...,...,...,...,...
0,亿纬锂能,-0.264644,NaN,NaN,10,0.044106,0.015,-0.268309,1
0,天合光能,-0.273533,NaN,NaN,3,0.044106,0.002,-0.273578,1
0,天合光能,-0.273533,NaN,NaN,3,0.044106,0.003,-0.273578,1
0,天合光能,-0.273533,NaN,NaN,3,0.044106,0.004,-0.273578,1


In [19]:
writer = pd.ExcelWriter('lstm_output_final.xlsx', engine='xlsxwriter')
trading_record.to_excel(writer, sheet_name='Sheet1', index=False)
writer.save()